# 处理原始数据按cell存


In [15]:
def define_bins(genome_file, resolutions):
  '''Takes chromosome sizes, and a list of desired resolutions and defines the
    bins that cover the genome at the desired resolution(s)'''
  bins = {}
  valid_chroms = {}
  for resolution in resolutions:
    bins[resolution] = {}
  for resolution in resolutions:
    for line in chromsizes:
      hindex = 0
      chromname, length = line.split()
      valid_chroms[chromname] = True
      for i in range(0, int(length), resolution):
        bins[resolution][(chromname, i)] = hindex
        hindex += 1
  return bins, valid_chroms

def cell_sort(percentages):

  '''Iterate through the percentages file generated by the scHi-C pipeline, and return a list
      of cell barcodes where the cell has >=1000 unique reads, passes cistrans_cutoff and is 0.95 one species'''
  cells = {}
  for line in percentages:
    hup, mup, huc, muc, val, tot, bc1, bc2, sig, length, ones, twos, threes, fours, ratio, gtype, celltype = line.split()
    if length == 'Long': continue
    if sig == 'Randomized': continue
    if float(hup) >= 0.95 and int(huc) >= 5000 and float(ratio) > 1:
      identity = ('human', huc, celltype)
      barcode = bc1 + '-' + bc2
      cells[barcode] = identity # coverage
  return cells

def validpair_walk(validpairs, cell_list, resolutions, bins, valid_chroms):
  '''bedpe_walk walks through a validpairs file, splits out reads to outfiles 
  of the format bin1<t>bin2<t>count<t>norm_count<t>chrom1<t>chrom2'''
  cell_index = 1
  cell_matrices = {}
  for resolution in resolutions:
    cell_matrices[resolution] = {}
  for line in validpairs:
    n1, f1, r1, n2, f2, r2, name, q1, a2, s1, s2, bc1, bc2, frag1, dist1, frag2, dist2 = line.split()
    species1 = n1.split('_')[0]
    species2 = n2.split('_')[0]
    barcode = bc1 + '-' + bc2 
    if barcode in cell_list:
      if species1 != species2: continue
      if species1 != cell_list[barcode][0]: continue
      if n1 not in valid_chroms: continue
      if n2 not in valid_chroms: continue
      for resolution in resolutions:
        if barcode not in cell_matrices[resolution]:
          cell_matrices[resolution][barcode] = Counter()
        pos1_reduce_int = (int(f1)+int(r1))/2 // resolution * resolution
        pos2_reduce_int = (int(f2)+int(r2))/2 // resolution * resolution
        bin1 = bins[resolution][(n1, pos1_reduce_int)]
        bin2 = bins[resolution][(n2, pos2_reduce_int)]
        if bin1 <= bin2:
          key = (bin1, bin2, n1, n2)
        else:
          key = (bin2, bin1, n2, n1)
        cell_matrices[resolution][barcode][key] += 1
  return cell_matrices
          

def normalizeMatrix(matrix):
  '''Given a matrix, root normalize the values in the matrix'''
  cov = Counter()
  normed = {}
  for i in matrix:
    bin1, bin2, chrom1, chrom2 = i
    count = matrix[i]
    cov[bin1] += count
    cov[bin2] += count
  for i in matrix:
    bin1, bin2, chrom1, chrom2 = i
    normed[i] = float(matrix[i]) / sqrt(cov[bin1]) / sqrt(cov[bin2])
  return normed
  
def filter_low_quality_cells(matrices, resolutions):

  min_chrom_contacts = {}
  for line in chromsizes:
    chromname, length = line.split()
    min_chrom_contacts[chromname] = int(length) // 1000000
  print('min_chrom_contacts:', min_chrom_contacts)
  del_barcodes = {}
  for resolution in resolutions:
    del_barcodes[resolution] = []
    for barcode in matrices[resolution]:
      nondiagonal_contacts = 0
      chrom_contacts = {}
      for key in matrices[resolution][barcode]:
        bin1, bin2, chrom1, chrom2 = key
        count = matrices[resolution][barcode][key]
        if bin1 != bin2 or chrom1 != chrom2:
          nondiagonal_contacts += count
        if chrom1 == chrom2:      
          if chrom1 not in chrom_contacts:
            chrom_contacts[chrom1] = count
          else:
            chrom_contacts[chrom1] += count
      high_quality = True
      if nondiagonal_contacts <= 5000:
        high_quality = False
      if len(chrom_contacts) != 23: #破案了，有一个chr22压根没有而不是太少，所以直接漏掉了,所以加这个
        if barcode == 'GCCTTAGG-TGAGGCAA':
          print('find you23333')
        high_quality = False
      for item in chrom_contacts.items(): 
        chrom, contacts = item
        if contacts < min_chrom_contacts[chrom]:
          high_quality = False
      if high_quality == False:
        del_barcodes[resolution].append(barcode)

    for barcode in del_barcodes[resolution]:
      del matrices[resolution][barcode]
  return matrices
      
    
import os,sys,re
from collections import Counter
from math import sqrt

chromsizes = open('/content/drive/MyDrive/毕设/data/Ramani/human_chromsize.txt').readlines()
percentages = open('/content/drive/MyDrive/毕设/data/Ramani/下载的原始数据集文件/GSM2254215_ML1.percentages.txt').readlines()
validpairs = open('/content/drive/MyDrive/毕设/data/Ramani/下载的原始数据集文件/GSM2254215_ML1.validPairs.txt').readlines() # 过滤前后474->438
# percentages = open('/content/drive/MyDrive/毕设/data/Ramani/下载的原始数据集文件/GSM2254217_ML3.percentages.txt').readlines()
# validpairs = open('/content/drive/MyDrive/毕设/data/Ramani/下载的原始数据集文件/GSM2254217_ML3.validPairs.txt').readlines() # 过滤前后156->55
resolutions = [1000000]

# 按分辨率切成bin
bins, valid_chroms = define_bins(chromsizes, resolutions)

# 用percentage文件初步筛选有效的barcode
cell_list = cell_sort(percentages)

# 用validpairs文件处理细胞信息了
cell_matrices = validpair_walk(validpairs, cell_list, resolutions, bins, valid_chroms)

print(len(cell_matrices[1000000].items()))
# 根据论文筛选高质量细胞
cell_matrices = filter_low_quality_cells(cell_matrices, resolutions)
print(len(cell_matrices[1000000].items()))

for resolution in resolutions:
  for barcode in cell_matrices[resolution]:
    fho_name = "cell_matrix_data/%s_%s_%s.matrix" % (cell_list[barcode][2], barcode, resolution)
    fho = open(fho_name, 'w')
    norm = normalizeMatrix(cell_matrices[resolution][barcode])
    for i in norm:
      fho.write("%s\t%s\t%s\t%s\t%s\t%s\n" % (i[0], i[1], cell_matrices[resolution][barcode][i], norm[i], i[2], i[3]))
    fho.close() 

474
min_chrom_contacts: {'human_chr1': 249, 'human_chr2': 243, 'human_chr3': 198, 'human_chr4': 191, 'human_chr5': 180, 'human_chr6': 171, 'human_chr7': 159, 'human_chr8': 146, 'human_chr9': 141, 'human_chr10': 135, 'human_chr11': 135, 'human_chr12': 133, 'human_chr13': 115, 'human_chr14': 107, 'human_chr15': 102, 'human_chr16': 90, 'human_chr17': 81, 'human_chr18': 78, 'human_chr19': 59, 'human_chr20': 63, 'human_chr21': 48, 'human_chr22': 51, 'human_chrX': 155}
find you23333
438


# 处理后数据统计与检查


In [16]:
import os
file_list = os.listdir('./cell_matrix_data')

num_1000000_K562 = 0
num_1000000_GM12878 = 0
num_1000000_HeLa = 0
num_1000000_HAP1 = 0

for name in file_list:
  if '1000000' in name and 'K562' in name:
    num_1000000_K562 += 1
  elif '1000000' in name and 'GM12878' in name:
    num_1000000_GM12878 += 1
  elif '1000000' in name and 'HeLa' in name:
    num_1000000_HeLa += 1
  elif '1000000' in name and 'HAP1' in name:
    num_1000000_HAP1 += 1
  else:
    continue
print(len(file_list))
print(num_1000000_HeLa)
print(num_1000000_HAP1)
print(num_1000000_GM12878)
print(num_1000000_K562)

493
245
193
12
43


In [17]:
# 读cell_matrix_data文件名得到所有细胞
import os
import numpy as np

file_list = os.listdir('./cell_matrix_data')
f = open('all_cells.txt','w')
for name in file_list:
  f.write(name[:-15] + '\n')
f.close()

In [20]:
file_list = os.listdir('./cell_matrix_data')
labels = []
for name in file_list:
  labels.append(name.split(sep='_')[0])
np.save('cell_label.npy', labels)
